# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

## Dataset

### Overview
We wil predict median house valie from kaggle's data set "Calofornia housing". Data set was downloaded as a csv and uploaded i.e. registered as data set in the workspace. 


### Workspace Set Up and Initialize an Experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FNBTELTVE to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-126995
aml-quickstarts-126995
southcentralus
502bce8d-d66b-4661-a423-f21e61d20123


In [5]:
os.listdir(os.curdir), os.getcwd()

(['.ipynb_aml_checkpoints', 'automl.ipynb'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook126995/code/Users/odl_user_126995')

In [6]:
# choose a name for experiment
experiment_name = 'automl_housing'
project_folder = './automl-housing'
os.makedirs(project_folder, exist_ok=True)

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl_housing,quick-starts-ws-126995,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
os.listdir(os.curdir)

['.ipynb_aml_checkpoints', 'automl-housing', 'automl.ipynb']

In [ ]:
#files and directories
cc=os.getcwd
os.listdir(os.curdir)

### Create or Attach a Compute Resource

In [8]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook126995"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Data set

In [9]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "housing"
description_text = "California Housing Dataset for Udacity project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = ''
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


data = dataset.to_pandas_dataframe()
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [13]:
import pandas as pd

def prepare_data(data):    
    encoded_column=pd.get_dummies(data['ocean_proximity'], prefix='ocp')
    data=data.join(encoded_column)
    data=data.drop("ocean_proximity", axis=1)

    train, test = train_test_split(data, test_size=0.2, random_state=42)
    return train, test

train, test = prepare_data(data)
type(train), type(test), train.shape, test.shape

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 (16512, 14),
 (4128, 14))

In [14]:
columns=train.columns
train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False)

In [15]:
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

Uploading an estimated of 1 files
Uploading trainset.csv
Uploaded trainset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [16]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

Uploading an estimated of 1 files
Uploading testset.csv
Uploaded testset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [17]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

## AutoML Configuration

Since we are trying to predict a numeric value, in particular, "median_house_values", we have to define the task as "regression". Featurization is set to "auto" and we enable early stopping to avoid overfitting. We pass in our train and test sets and choose "normalized_mean_absolute_error" as the primary metric.

In [35]:
azureml.train.automl.utilities.get_primary_metrics("regression")

['spearman_correlation',
 'r2_score',
 'normalized_root_mean_squared_error',
 'normalized_mean_absolute_error']

In [39]:
# TODO: Put your automl settings here
# TODO: Put your automl config here

automl_setting={
    "featurization": "auto",
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "compute_target": compute_target
}

task="regression" 
automl_config = AutoMLConfig( 
    task=task, 
    primary_metric='normalized_root_mean_squared_error', 
    training_data=train, 
    validation_data = test, 
    label_column_name='median_house_value', 
    **automl_setting
)

In [40]:
# TODO: Submit your experiment
from azureml.widgets import RunDetails

automl_run = experiment.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

Running on remote.
Running on remote compute: notebook126995
Parent Run ID: AutoML_87e2c6b2-2888-4320-a418-5c087560a1fb

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

***************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [41]:
automl_run.wait_for_completion()

{'runId': 'AutoML_87e2c6b2-2888-4320-a418-5c087560a1fb',
 'target': 'notebook126995',
 'status': 'Completed',
 'startTimeUtc': '2020-11-20T11:32:08.512937Z',
 'endTimeUtc': '2020-11-20T12:02:27.743642Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook126995',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_housing","subscription_id":"502bce8d-d66b-4661-a423-f21e61d20123","resource_group":"aml-quickstarts-126995","workspace_name":"quick-starts-ws-126995","region":"southcentralus","compute_target":"notebook126995","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"vali

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [42]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "automl_model_housing.joblib")

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

['automl_model_housing.joblib']

In [44]:
automl_run_metrics = automl_run.get_metrics()
automl_run_metrics

{'normalized_mean_absolute_error': 0.06675515129322772,
 'r2_score': 0.8228235321116265,
 'mean_absolute_error': 32376.38188751803,
 'explained_variance': 0.8228777634621539,
 'root_mean_squared_error': 48184.39936887613,
 'root_mean_squared_log_error': 0.23883385833995038,
 'median_absolute_error': 21358.68757477193,
 'normalized_root_mean_squared_log_error': 0.06811054968772484,
 'mean_absolute_percentage_error': 18.38221463878435,
 'spearman_correlation': 0.9138951035775631,
 'normalized_root_mean_squared_error': 0.09934886736317816,
 'normalized_median_absolute_error': 0.04403834948056282,
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_87e2c6b2-2888-4320-a418-5c087560a1fb_0/predicted_true',
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_87e2c6b2-2888-4320-a418-5c087560a1fb_0/residuals'}

In [46]:
print('Bets Run ID', automl_run.id)
print('\n MAE', automl_run_metrics['mean_absolute_error'])

Bets Run ID AutoML_87e2c6b2-2888-4320-a418-5c087560a1fb_0

 MAE 32376.38188751803
